# Seer-py Example
This workbook runs through a brief example of how to use the seer-py library to interact directly with the seer database to view what data exists, download raw data, and upload new labels.

Firstly, we need to load in seer-py as well as data-handling tools numpy and pandas

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
import seer
from seer.plotting import seerPlot

## Client
Now that we have the libraries loaded, we want to interact with the database. To do this, we need to create a client. The client handles the connection to the database as well as user crendentials. `createClient()` will prompt user for a username and password, and verify these credentials with the database.

In [3]:
client = seer.SeerConnect()

Login Successful


## Metadata
Next, we want to see all the data that is available. This is handled by `createMetaData` which will query the database to see everything that is available to the user, and return a list of pandas DataFrames. These dataframes will then be used to download data.

In [5]:
allData, channelGroups, segments, channels, dataChunks, labelGroups, labels = client.createMetaData()
# OR equivalently:
# allData = client.createMetaData()[0]

studyId = 'study2'
allData = allData[allData['id']==studyId]
allData.head()

Exception: {'statusCode': 500, 'errorCode': 'SERVER_ERROR', 'message': 'Internal server error'}

###  
Now that we have meta data, we can filter down to a specific study, patient, data type, etc. In this example, we want eeg data from Study 2, and we use pandas filtering methods to reduce `allData` down to the rows of interest

In [ ]:
studyId = 'study2'
allData = allData[allData['id']==studyId]
allData = allData[allData['channelGroups.name'].str.contains('EEG')]
allData.head(5)

###  
`getLinks` takes a pandas dataframe (as given by `createMetaData`) and downloads raw data, returning another dataframe with raw data, a timestamp, and study metadata, such as id, channelGroup, and segment 

In [ ]:
data = client.getLinks(allData.copy())
data.head()

####  
To see what we are actually looking at, lets make a plot. `seerPlot` is a simple function to plot either a pandas dataframe or numpy array.

In [ ]:
start = 0
end = 1280
seerPlot(x=data.iloc[start:end, 7], figSize=(15,2))

####  
Now we want to create a new label group, and if it exists, we will delete and re-create it.
`delLabelGroup` will delete a label group given a labelGroupId
`addLabelGroup` will create a new labelGroup given studyId, labelTypeId, labelGroupName, labelGroupDescription

In [ ]:
labelGroupName = 'testname1'
labelGroupDescription = 'testDescription'
if len(allData[(allData['labelGroups.name']==labelGroupName) & (allData['id']==studyId)])!=0:
    delLG = client.delLabelGroup(allData[(allData['labelGroups.name']==labelGroupName) & (allData['id']==studyId)]['labelGroups.id'].iloc[0])
labelGroup = client.addLabelGroup(studyId, labelGroupName, labelGroupDescription)

Now we're ready to create our new label. In this example, we will highlight any areas where the amplitide is above 90% over a one second window. This could be used to highlight areas of extreme amplitude, which are potentially artifacts in the recording.
`alarm` is an array of start and finish times (millisecods since epoch) which have amplitude above 90%
The function `makeLabel` takes an array of ones and zeros (1 for true label and 0 for false) and creates a start time-finish time array.

In [ ]:
dataNew = client.applyMovAvg(data.iloc[:,7].as_matrix(), 64)
alarmLabel = np.any(dataNew>np.percentile(dataNew, q=90), axis=1)
alarm = client.makeLabel(alarmLabel, data['time'].values)
print(alarm[:5, :])

Now we can upload the labels to the Seer database with `addLabel`

In [ ]:
for i in range(alarm.shape[0]):
    client.addLabel(labelGroup, alarm[i,0], alarm[i,1]-alarm[i,0], 11.0)

In [ ]:
start = 0
end = 1280
seerPlot(x=data.iloc[start:end, 7], y=alarmLabel[start:end], figSize=(15,2))

## Getting more Information

If you're not sure how something works, try the function name, followed by '?' (e.g. seer.addLabel?), or help(function_name)

In [ ]:
help(client.addLabel)